In [1]:
import sympy
from cmath import pi
from kinematicBuilder import *

# Two-Link Planar 

## Examples

In [2]:
twp = TwoLinkPlanar(2,3)
twp.directKinSymbols()

Matrix([
[2*cos(q_1) + 3*cos(q_1 + q_2)],
[2*sin(q_1) + 3*sin(q_1 + q_2)],
[                            0],
[                    q_1 + q_2],
[                            0],
[                            0]])

In [3]:
twp.directLambda(pi/2,pi/2)

array([[-1.0000000e+00, -1.2246468e-16,  0.0000000e+00, -3.0000000e+00],
       [ 1.2246468e-16, -1.0000000e+00,  0.0000000e+00,  2.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [4]:
twp.inverseLambda(-1,2)

array([[0.46364761],
       [2.30052398]])

# Two-Link Planar on rotatory base

## Examples

In [5]:
tlb = TwoLinkAndBase(50,50)
tlb.directKinHomoTSymbols()

Matrix([
[cos(q_1)*cos(q_2 + q_3), -sin(q_1), sin(q_2 + q_3)*cos(q_1), (50*cos(q_2) + 50*cos(q_2 + q_3))*cos(q_1)],
[sin(q_1)*cos(q_2 + q_3),  cos(q_1), sin(q_1)*sin(q_2 + q_3), (50*cos(q_2) + 50*cos(q_2 + q_3))*sin(q_1)],
[        -sin(q_2 + q_3),         0,          cos(q_2 + q_3),            50*sin(q_2) + 50*sin(q_2 + q_3)],
[                      0,         0,                       0,                                          1]])

In [6]:
tlb.inverseKinSymbols()

Matrix([
[                                                                                                                 atan2(y, x)],
[atan2(z, sqrt(x**2 + y**2)) - atan2(50*sqrt(1 - (x**2/5000 + y**2/5000 + z**2/5000 - 1)**2), x**2/100 + y**2/100 + z**2/100)],
[                                                                                 acos(x**2/5000 + y**2/5000 + z**2/5000 - 1)]])

In [7]:
tlb.directLambda(0,pi/4,pi/4)

array([[ 6.12323400e-17, -0.00000000e+00,  1.00000000e+00,
         3.53553391e+01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00,  6.12323400e-17,
         8.53553391e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [8]:
tlb.inverseLambda(4,0,4)

array([[ 0.        ],
       [-0.72879941],
       [ 3.02839514]])

# 3DOF Spherical wrist 

## Examples

In [9]:
rw = SphericalWrist("xyx")

In [10]:
rw.directKinHomoTSymbols()

Matrix([
[          cos(q_2),                               sin(q_2)*sin(q_3),                               sin(q_2)*cos(q_3), 0],
[ sin(q_1)*sin(q_2), -sin(q_1)*sin(q_3)*cos(q_2) + cos(q_1)*cos(q_3), -sin(q_1)*cos(q_2)*cos(q_3) - sin(q_3)*cos(q_1), 0],
[-sin(q_2)*cos(q_1),  sin(q_1)*cos(q_3) + sin(q_3)*cos(q_1)*cos(q_2), -sin(q_1)*sin(q_3) + cos(q_1)*cos(q_2)*cos(q_3), 0],
[                 0,                                               0,                                               0, 1]])

In [11]:
rw.inverseKinSymbols()

Matrix([
[atan2(R[1, 0], -R[2, 0])],
[           acos(R[0, 0])],
[ atan2(R[0, 1], R[0, 2])]])

In [12]:
# rw.inverse(0,0,0)

# Decoupled kinematics 6DOF

## Examples

In [13]:
d6d = Decoupled6DOF(2,3)
d6d.directKinHomoTSymbols()

Matrix([
[                                                                         sin(q_1)*sin(q_2)*sin(q_3)*cos(q_2 + q_3) - sin(q_2)*sin(q_2 + q_3)*cos(q_3) + cos(q_1)*cos(q_2)*cos(q_2 + q_3),                                        -sin(q_1)*cos(q_2) + sin(q_2)*sin(q_3)*cos(q_1),                                                                          sin(q_1)*sin(q_2)*sin(q_3)*sin(q_2 + q_3) + sin(q_2)*cos(q_3)*cos(q_2 + q_3) + sin(q_2 + q_3)*cos(q_1)*cos(q_2),                                                                          (2*sin(q_2) + 3*sin(q_2 + q_3))*sin(q_2)*cos(q_3) + (2*cos(q_2) + 3*cos(q_2 + q_3))*sin(q_1)*sin(q_2)*sin(q_3) + (2*cos(q_2) + 3*cos(q_2 + q_3))*cos(q_1)*cos(q_2)],
[(-sin(q_1)*sin(q_3)*cos(q_2) + cos(q_1)*cos(q_3))*sin(q_1)*cos(q_2 + q_3) - (-sin(q_1)*cos(q_2)*cos(q_3) - sin(q_3)*cos(q_1))*sin(q_2 + q_3) + sin(q_1)*sin(q_2)*cos(q_1)*cos(q_2 + q_3),      (-sin(q_1)*sin(q_3)*cos(q_2) + cos(q_1)*cos(q_3))*cos(q_1) - sin(q_1)**2*sin(q_2), (-sin(q_1)*sin(q_

In [14]:
d6d.inverseKinSymbols()

Matrix([
[                                                                                                                                                                           atan2(y, x)],
[                                                             atan2(z, sqrt(x**2 + y**2)) - atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                                                                                                             acos(x**2/12 + y**2/12 + z**2/12 - 13/12)],
[                                                                                                                                               -atan2(y, x) + atan2(R[1, 0], -R[2, 0])],
[-acos(x**2/12 + y**2/12 + z**2/12 - 13/12) + acos(R[0, 0]) - atan2(z, sqrt(x**2 + y**2)) + atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                            

In [15]:
# d6d.inverse(1.5,1.5,0,0,pi/2,0)

# Kineamtic tryouts

In [16]:
Lv = 0.5
L1 = sympy.Symbol('L_1')
L2 = sympy.Symbol('L_2')
rotM = sympy.MatrixSymbol('R',4,4)

twoLinksBase = TwoLinkAndBase()

r = sympy.sqrt(twoLinksBase.x**2 + twoLinksBase.y**2)
r_p = sympy.simplify(r*sympy.atan2(0.0425,r).evalf())
r_p


sqrt(x**2 + y**2)*atan2(0.0425, sqrt(x**2 + y**2))

In [17]:
angleOffset = sympy.atan2(0.02,Lv).evalf()
Lv_p = Lv/sympy.cos(angleOffset).evalf()
Lv_p

0.500399840127872

In [18]:

twoLinksBase.setSymbols(Lv,Lv_p,x=rotM[0,3], y=rotM[1,3], z=rotM[2,3],r_expr=r_p)
twoLinksBase.inverseSym


Matrix([
[                                                                                                                                                                                                                                                                                                       atan2(R[1, 3], R[0, 3])],
[-atan2(sqrt(313/1250 - ((R[0, 3]**2 + R[1, 3]**2)*atan2(17/400, sqrt(R[0, 3]**2 + R[1, 3]**2))**2 + R[2, 3]**2 - 1251/2500)**2), (R[0, 3]**2 + R[1, 3]**2)*atan2(17/400, sqrt(R[0, 3]**2 + R[1, 3]**2))**2 + R[2, 3]**2 - 1/2500) + atan2(R[2, 3], sqrt(R[0, 3]**2 + R[1, 3]**2)*atan2(17/400, sqrt(R[0, 3]**2 + R[1, 3]**2)))],
[                                                                                                                                                acos(1313575924195*(R[0, 3]**2 + R[1, 3]**2)*atan2(17/400, sqrt(R[0, 3]**2 + R[1, 3]**2))**2/657313182463 + 1313575924195*R[2, 3]**2/657313182463 - 845623115460/845622845293)]])

In [19]:
twoLinksBase.directKinHomoTSymbols()

Matrix([
[cos(q_1)*cos(q_2 + q_3), -sin(q_1), sin(q_2 + q_3)*cos(q_1), (0.5*cos(q_2) + 0.500399840127872*cos(q_2 + q_3))*cos(q_1)],
[sin(q_1)*cos(q_2 + q_3),  cos(q_1), sin(q_1)*sin(q_2 + q_3), (0.5*cos(q_2) + 0.500399840127872*cos(q_2 + q_3))*sin(q_1)],
[        -sin(q_2 + q_3),         0,          cos(q_2 + q_3),            0.5*sin(q_2) + 0.500399840127872*sin(q_2 + q_3)],
[                      0,         0,                       0,                                                          1]])

In [20]:
Rotations().eulerToMatrixSequenceSym("xyz")

Matrix([
[                       cos(b)*cos(c),                        -sin(c)*cos(b),         sin(b)],
[sin(a)*sin(b)*cos(c) + sin(c)*cos(a), -sin(a)*sin(b)*sin(c) + cos(a)*cos(c), -sin(a)*cos(b)],
[sin(a)*sin(c) - sin(b)*cos(a)*cos(c),  sin(a)*cos(c) + sin(b)*sin(c)*cos(a),  cos(a)*cos(b)]])

In [21]:
Rotations().eulerToMatrixSequenceSym("zyx")

Matrix([
[cos(a)*cos(b), -sin(a)*cos(c) + sin(b)*sin(c)*cos(a), sin(a)*sin(c) + sin(b)*cos(a)*cos(c)],
[sin(a)*cos(b),  sin(a)*sin(b)*sin(c) + cos(a)*cos(c), sin(a)*sin(b)*cos(c) - sin(c)*cos(a)],
[      -sin(b),                         sin(c)*cos(b),                        cos(b)*cos(c)]])

In [22]:
Rotations().matrixToEulerSequence("zyx",Rotations().rotMatrixX(pi/2))

array([[0],
       [0],
       [1.57079632679490]], dtype=object)

# Screw Theory

In [23]:
transform = np.array((
    ( 1, 0, 0, 1.1),
    ( 0, 0,-1,-0),
    ( 0, 1, 0, 0),
    ( 0, 0, 0, 1)
))
adj = Rotations().adjointT(transform)
adj*sympy.Matrix([0,1,0,0,0,0])


Matrix([
[  0],
[  0],
[1.0],
[  0],
[  0],
[  0]])

Screw from twist
- Screw axis: $\hat \xi = \frac{\omega}{||\omega||} = \hat \omega$ 
- Screw rate value: $ \dot \theta = ||\omega||$ 
- Pitch: $h = \frac{\hat \omega^T v}{\dot \theta} =\frac{\hat \omega^T v}{||\omega||^2} $

In [46]:
omegaSym = sympy.MatrixSymbol('omega', 3, 1)
vSym     = sympy.MatrixSymbol('v', 3, 1)
omega = sympy.Matrix(omegaSym)
v     = sympy.Matrix(vSym)


dtheta = sympy.sqrt(omega.dot(omega))
xi = omega/dtheta
h = (xi.T * v)/dtheta
q = (xi.cross(v))/dtheta
# q
screw = sympy.Matrix(sympy.BlockMatrix([
    [xi],
    [v/dtheta]
]))
sympy.simplify(q)

Matrix([
[ (omega[1, 0]*v[2, 0] - omega[2, 0]*v[1, 0])/(omega[0, 0]**2 + omega[1, 0]**2 + omega[2, 0]**2)],
[(-omega[0, 0]*v[2, 0] + omega[2, 0]*v[0, 0])/(omega[0, 0]**2 + omega[1, 0]**2 + omega[2, 0]**2)],
[ (omega[0, 0]*v[1, 0] - omega[1, 0]*v[0, 0])/(omega[0, 0]**2 + omega[1, 0]**2 + omega[2, 0]**2)]])

In [25]:
twist = sympy.Matrix(sympy.BlockMatrix([
    [omega],
    [v]
]))
twist

Matrix([
[omega[0, 0]],
[omega[1, 0]],
[omega[2, 0]],
[    v[0, 0]],
[    v[1, 0]],
[    v[2, 0]]])

In [26]:
omega_val = sympy.Matrix([ 0, 2, 2])
v_val     = sympy.Matrix([ 4, 0, 0])

subs = {
    omegaSym:omega_val,
    vSym:v_val
}
dtheta.subs(subs)
xi.subs(subs)
h.subs(subs)
q.subs(subs)
# screw.subs(subs).evalf()


Matrix([
[ 0],
[ 1],
[-1]])

## Adjoint representation of HMT
https://www.mecharithm.com/velocities-in-robotics-angular-velocities-twists/

https://publish.illinois.edu/ece470-intro-robotics/files/2019/09/09-lecture.pdf

https://www.researchgate.net/publication/328318823_Screw_Theory_for_Robotics_-_A_practical_approach_for_Modern_Robot_KINEMATICS_-_An_Illustrated_Handbook

In [27]:
Rsym = sympy.MatrixSymbol('R',3,3)
psym = sympy.MatrixSymbol('p',3,1)
R = sympy.Matrix(Rsym)
p = sympy.Matrix(psym)
skewP = sympy.Matrix([
            [ 0     ,-p[2,0], p[1,0]],
            [ p[2,0], 0     ,-p[0,0]],
            [-p[1,0], p[0,0], 0     ]
        ])
adjT = sympy.Matrix(sympy.BlockMatrix([
    [R      , sympy.zeros(3)],
    [skewP*R, R             ]
]))
adjTinv = sympy.Matrix(sympy.BlockMatrix([
    [R.T       , sympy.zeros(3)],
    [-R.T*skewP, R.T           ]
]))
# adjT = sympy.Matrix(sympy.BlockMatrix([
#     [R             , skewP*R],
#     [sympy.zeros(3), R      ]
# ]))
adjT

Matrix([
[                           R[0, 0],                            R[0, 1],                            R[0, 2],       0,       0,       0],
[                           R[1, 0],                            R[1, 1],                            R[1, 2],       0,       0,       0],
[                           R[2, 0],                            R[2, 1],                            R[2, 2],       0,       0,       0],
[-R[1, 0]*p[2, 0] + R[2, 0]*p[1, 0], -R[1, 1]*p[2, 0] + R[2, 1]*p[1, 0], -R[1, 2]*p[2, 0] + R[2, 2]*p[1, 0], R[0, 0], R[0, 1], R[0, 2]],
[ R[0, 0]*p[2, 0] - R[2, 0]*p[0, 0],  R[0, 1]*p[2, 0] - R[2, 1]*p[0, 0],  R[0, 2]*p[2, 0] - R[2, 2]*p[0, 0], R[1, 0], R[1, 1], R[1, 2]],
[-R[0, 0]*p[1, 0] + R[1, 0]*p[0, 0], -R[0, 1]*p[1, 0] + R[1, 1]*p[0, 0], -R[0, 2]*p[1, 0] + R[1, 2]*p[0, 0], R[2, 0], R[2, 1], R[2, 2]]])

In [28]:
tsym = sympy.MatrixSymbol('t',3,1)
osym = sympy.MatrixSymbol('o',3,1)
lin = sympy.Matrix(tsym)
ang = sympy.Matrix(osym)
twist = sympy.Matrix(sympy.BlockMatrix([
    [ang],
    [lin]
]))
twist

Matrix([
[o[0, 0]],
[o[1, 0]],
[o[2, 0]],
[t[0, 0]],
[t[1, 0]],
[t[2, 0]]])

In [29]:
q = adjT*twist
# q = adjTinv*twist
q

Matrix([
[                                                                                                                                             R[0, 0]*o[0, 0] + R[0, 1]*o[1, 0] + R[0, 2]*o[2, 0]],
[                                                                                                                                             R[1, 0]*o[0, 0] + R[1, 1]*o[1, 0] + R[1, 2]*o[2, 0]],
[                                                                                                                                             R[2, 0]*o[0, 0] + R[2, 1]*o[1, 0] + R[2, 2]*o[2, 0]],
[(-R[1, 0]*p[2, 0] + R[2, 0]*p[1, 0])*o[0, 0] + (-R[1, 1]*p[2, 0] + R[2, 1]*p[1, 0])*o[1, 0] + (-R[1, 2]*p[2, 0] + R[2, 2]*p[1, 0])*o[2, 0] + R[0, 0]*t[0, 0] + R[0, 1]*t[1, 0] + R[0, 2]*t[2, 0]],
[   (R[0, 0]*p[2, 0] - R[2, 0]*p[0, 0])*o[0, 0] + (R[0, 1]*p[2, 0] - R[2, 1]*p[0, 0])*o[1, 0] + (R[0, 2]*p[2, 0] - R[2, 2]*p[0, 0])*o[2, 0] + R[1, 0]*t[0, 0] + R[1, 1]*t[1, 0] + R[1, 2]*t[2, 0]],
[(-R[0, 0]*

In [30]:
Rotations().rotMatrixX(-pi/2)

Matrix([
[1.0,                    0,                    0],
[  0, 6.12323399573677e-17,                  1.0],
[  0,                 -1.0, 6.12323399573677e-17]])

In [31]:
subs = {
    Rsym: sympy.Matrix([
        [ 1, 0, 0],
        [ 0, 0, 1],
        [ 0,-1, 0]]),
    psym: sympy.Matrix([[3],[0],[0]]),

    # tsym: sympy.Matrix([[1.0],[1.1],[1.2]]),
    # osym: sympy.Matrix([[1.3],[1.4],[1.5]])
    tsym: sympy.Matrix([[1],[0],[0]]),
    osym: sympy.Matrix([[0],[0],[2]])

}
q = adjTinv*twist
q.subs(subs)
# adjT.subs(subs)

Matrix([
[ 0],
[-2],
[ 0],
[ 1],
[ 0],
[ 6]])

In [32]:
"""
        Z
        ^ Y
    Y   |/
    ^   o -- > X
    |       
    o -- > X
   /
  Z
"""
subs = {
    Rsym: sympy.Matrix([
        [ 1, 0, 0],
        [ 0, 0, 1],
        [ 0,-1, 0]]),
    psym: sympy.Matrix([[2*sympy.cos(pi/4)],[2*sympy.sin(pi/4)],[0]]),

    # tsym: sympy.Matrix([[1.0],[1.1],[1.2]]),
    # osym: sympy.Matrix([[1.3],[1.4],[1.5]])
    tsym: sympy.Matrix([
        # [-2/sympy.sqrt(2)],
        # [2/sympy.sqrt(2)],
        # [0]
        [-2/sympy.sqrt(2)],
        [0],
        [2/sympy.sqrt(2)]
    ]),
    osym: sympy.Matrix([[0],[0],[1]])

}
q = adjT*twist
q.subs(subs).evalf()
# adjT.subs(subs)
# twist.subs(subs)

Matrix([
[               0],
[             1.0],
[               0],
[-1.4142135623731],
[ 1.4142135623731],
[ 1.4142135623731]])

In [33]:
"""
    Z                 Y
    ^ Y               ^
    |/                |
    o -- > X    =>    o -- > X
                     /
                    Z
"""
subs = {
    Rsym: sympy.Matrix([
        [ 1, 0, 0],
        [ 0, 0,-1],
        [ 0, 1, 0]]),
    psym: sympy.Matrix([[0.5],[0],[0]]),

    # tsym: sympy.Matrix([[1.0],[1.1],[1.2]]),
    # osym: sympy.Matrix([[1.3],[1.4],[1.5]])
    tsym: sympy.Matrix([
        [0],
        [0],
        [0]
    ]),
    osym: sympy.Matrix([[0],[0],[2]])
}
q = adjTinv*twist
q.subs(subs).evalf()
# adjT.subs(subs)
# twist.subs(subs)

Matrix([
[   0],
[ 2.0],
[   0],
[   0],
[   0],
[-1.0]])

In [34]:
o_new = Rsym*osym
t_new = Rsym*tsym + skewP*Rsym*osym
twist_new = sympy.Matrix(sympy.BlockMatrix([
    [o_new],
    [t_new]
]))

twist_new.subs(subs).evalf()

Matrix([
[   0],
[-2.0],
[   0],
[   0],
[   0],
[-1.0]])

## Matrix representation of twist

In [35]:
tsym = sympy.MatrixSymbol('t',3,1)
osym = sympy.MatrixSymbol('o',3,1)
lin = sympy.Matrix(tsym)
ang = sympy.Matrix(osym)
twist = sympy.Matrix(sympy.BlockMatrix([
    [ang],
    [lin]
]))
twist

Matrix([
[o[0, 0]],
[o[1, 0]],
[o[2, 0]],
[t[0, 0]],
[t[1, 0]],
[t[2, 0]]])

In [36]:
mTwist = sympy.Matrix(sympy.BlockMatrix([
        [skewP.subs(psym,osym), lin             ],
        [sympy.zeros(1,3)     , sympy.zeros(1)  ]
    ]
))
mTwist

Matrix([
[       0, -o[2, 0],  o[1, 0], t[0, 0]],
[ o[2, 0],        0, -o[0, 0], t[1, 0]],
[-o[1, 0],  o[0, 0],        0, t[2, 0]],
[       0,        0,        0,       0]])

In [37]:

hmt = sympy.Matrix(sympy.BlockMatrix([
        [R, p],
        [sympy.zeros(1,3), sympy.eye(1)]
    ]
))
hmtInv = sympy.Matrix(sympy.BlockMatrix([
        [R.T, -R.T*p],
        [sympy.zeros(1,3), sympy.eye(1)]
    ]
))
hmt
hmtInv

Matrix([
[R[0, 0], R[1, 0], R[2, 0], -R[0, 0]*p[0, 0] - R[1, 0]*p[1, 0] - R[2, 0]*p[2, 0]],
[R[0, 1], R[1, 1], R[2, 1], -R[0, 1]*p[0, 0] - R[1, 1]*p[1, 0] - R[2, 1]*p[2, 0]],
[R[0, 2], R[1, 2], R[2, 2], -R[0, 2]*p[0, 0] - R[1, 2]*p[1, 0] - R[2, 2]*p[2, 0]],
[      0,       0,       0,                                                    1]])

In [38]:
qhmt = hmt*mTwist*hmtInv
# qhmt = hmtInv*mTwist*hmt
qhmt

Matrix([
[(R[0, 0]*o[1, 0] - R[0, 1]*o[0, 0])*R[0, 2] + (-R[0, 0]*o[2, 0] + R[0, 2]*o[0, 0])*R[0, 1] + (R[0, 1]*o[2, 0] - R[0, 2]*o[1, 0])*R[0, 0], (R[0, 0]*o[1, 0] - R[0, 1]*o[0, 0])*R[1, 2] + (-R[0, 0]*o[2, 0] + R[0, 2]*o[0, 0])*R[1, 1] + (R[0, 1]*o[2, 0] - R[0, 2]*o[1, 0])*R[1, 0], (R[0, 0]*o[1, 0] - R[0, 1]*o[0, 0])*R[2, 2] + (-R[0, 0]*o[2, 0] + R[0, 2]*o[0, 0])*R[2, 1] + (R[0, 1]*o[2, 0] - R[0, 2]*o[1, 0])*R[2, 0], (R[0, 0]*o[1, 0] - R[0, 1]*o[0, 0])*(-R[0, 2]*p[0, 0] - R[1, 2]*p[1, 0] - R[2, 2]*p[2, 0]) + (-R[0, 0]*o[2, 0] + R[0, 2]*o[0, 0])*(-R[0, 1]*p[0, 0] - R[1, 1]*p[1, 0] - R[2, 1]*p[2, 0]) + (R[0, 1]*o[2, 0] - R[0, 2]*o[1, 0])*(-R[0, 0]*p[0, 0] - R[1, 0]*p[1, 0] - R[2, 0]*p[2, 0]) + R[0, 0]*t[0, 0] + R[0, 1]*t[1, 0] + R[0, 2]*t[2, 0]],
[(R[1, 0]*o[1, 0] - R[1, 1]*o[0, 0])*R[0, 2] + (-R[1, 0]*o[2, 0] + R[1, 2]*o[0, 0])*R[0, 1] + (R[1, 1]*o[2, 0] - R[1, 2]*o[1, 0])*R[0, 0], (R[1, 0]*o[1, 0] - R[1, 1]*o[0, 0])*R[1, 2] + (-R[1, 0]*o[2, 0] + R[1, 2]*o[0, 0])*R[1, 1] + (R[1, 1]*o

In [39]:
mSubs = {
    Rsym: sympy.Matrix([
        [ 0,-1, 0],
        [ 0, 0,-1],
        [ 1, 0, 0]]),
    psym: sympy.Matrix([[3],[0],[0]]),

    tsym: sympy.Matrix([[0],[0],[0]]),
    osym: sympy.Matrix([[0],[0],[2]])
}
qhmt.subs(mSubs)

Matrix([
[0, 0, -2,  0],
[0, 0,  0,  0],
[2, 0,  0, -6],
[0, 0,  0,  0]])

# Pruebas